In [1]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import geopandas as gpd

import json
import panel.widgets as pnw

import warnings
import hvplot

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

pn.extension()

In [2]:
import pandas as pd
import geopandas as gpd
import json
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
import panel as pn
pn.extension()

In [3]:
# Geo-data
world = gpd.read_file(geopandas.datasets.get_path("naturalearth_lowres"))

# Cause of death - data
url = 'https://raw.githubusercontent.com/Denze8/Data-Visualization-Project/main/Data/full_data_geo.csv'
data = pd.read_csv(url)
idf = data.interactive()
data = data.drop(columns = ['geometry', 'continent', 'Country.Territory'])

# 
geo_data = world.merge(data, how="left", left_on = ['iso_a3'], right_on = ['Code'])
geo_data['Year'] = geo_data['Year'].values.astype(int)

/var/folders/k6/14q6_b813yncx2fqg44h95_r0000gn/T/ipykernel_68094/2712541781.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(geopandas.datasets.get_path("naturalearth_lowres"))


In [4]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

In [5]:
def bokeh_plot_map(gdf, column=None, title=''):
    """Plot bokeh map from GeoJSONDataSource """
    
    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar = ColorBar(color_mapper = color_mapper, label_standoff = 8, width = 500, height = 20, 
                         location=(0,0), orientation='horizontal')

    tools = 'wheel_zoom,pan,reset'
    p = figure(title = title, height=400 , width=850, toolbar_location='right', tools=tools)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    #Add patch renderer to figure
    p.patches('xs','ys', source=geosource, fill_alpha=1, line_width=0.5, line_color='black',  
              fill_color={'field' :column , 'transform': color_mapper})
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    return p

In [6]:
p = bokeh_plot_map(geo_data.loc[geo_data['Year'] == 2019], 'Meningitis')

pn.pane.Bokeh(p)

Bokeh(figure)

In [34]:
def map_dash():

    from bokeh.models.widgets import DataTable
    map_pane = pn.pane.Bokeh(width = 500, height = 400)
    data_select = pnw.Select(name='Cause of Death', value = 'Meningitis', options= list(geo_data.columns)[9:-2])
    year_slider = pnw.IntSlider(value = 1990, start = 1990, end = 2019)
    bar = None
    def update_map(event):
        gdata = geo_data.loc[geo_data['Year'] == year_slider.value]
        map_pane.object = bokeh_plot_map(gdata, data_select.value)
        return 
    
    idf_ = idf[idf["Year"] >= year_slider]
    bar = idf_.hvplot.bar(x = 'Year', y = data_select.value)
    
    year_slider.param.watch(update_map,'value')
    year_slider.param.trigger('value')
    data_select.param.watch(update_map, 'value')
    
    app = pn.Column(
            pn.Row(
                pn.Column(
                    data_select, year_slider), 
                map_pane),
        bar)
    return app

map_dash()

Column
    [0] Row
        [0] Column
            [0] Select(name='Cause of Death', options=['Alzheimer.s.Disease.and....], value='Alzheimer.s.Disease.and.O...)
            [1] IntSlider(end=2019, start=1990, value=1990)
        [1] Bokeh(figure, align='auto', height=400, width=500)
    [1] Interactive(Interactive)

In [30]:
pn.pane.Bokeh(width = 500, height = 400)

Bokeh(None, height=400, sizing_mode='fixed', width=500)

In [ ]:
app